In [83]:
#try this with arcpy and pandas and see if it's possible
#might just be some spatial joins to contend with and then use pandas for everything else

import arcpy
from arcpy import env
import geopandas as gpd
from arcgis.features import GeoAccessor, GeoSeriesAccessor
import os
import pandas as pd
import zipfile
from urllib.request import urlopen
import rasterio

import sys
sys.path.append("..")

from src.features.custom_functions import *

%load_ext autoreload
%autoreload 2

pd.set_option('display.max_columns', None)


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [10]:
#input datasets
town_name = 'Revere'

datasets_dir = r'K:\DataServices\datasets'


gdb = r'K:\DataServices\Projects\Current_Projects\Climate_Change\MVP_MMC_CoolRoofs_MVP\ArcGIS\CoolRoofs_Analysis.gdb'
env.overwriteOutput = True
#arcpy.env.outputCoordinateSystem = arcpy.SpatialReference("NAD 1983 StatePlane Massachusetts FIPS 2001 (Meters)")
rooftops_layer_name = town_name + '_footprints_with_info'
rooftops_layer = gpd.read_file(gdb, layer=rooftops_layer_name)

#municipalities
munis_fp = os.path.join(datasets_dir, "Boundaries\Spatial\mapc_towns_poly.shp")
munis = gpd.read_file(munis_fp)

In [79]:
#get shapefile from a massgis link
shapefile_excel = 'https://www.mass.gov/doc/massgis-parcel-data-download-links-table/download'
#shapefile_lookup = pd.read_excel(shapefile_excel)
#Actually maybe try that later, for now we can repurpose the 3A scripts and use data from the K drive

#repurpose 3A script to get parcels with MAPC Land Parcel Database info attached
muni_parcels = get_landuse_data(town_name)


In [39]:
#muni_w_structures = parcel_process(muni_parcels, rooftops_layer)

In [106]:
parcel_data = muni_parcels.copy()
structures_data = rooftops_layer.clip(parcel_data)
heat_fp = os.path.join(datasets_dir, 'Environment and Energy\Land_Surface_Temperature\Shapefile_LSTIndex\LSTindex.tif')


#reformat use code to match with real estate lookup codes
land_use_lookup['USE_CODE'] = land_use_lookup['USE_CODE'].apply(lambda x: '{0:0>3}'.format(x))
real_estate_lookup['use_code_3dg'] = real_estate_lookup['use_code_3dg'].str.replace('"', '') 

## JOIN PARCELS, CODES, AND STRUCTURES ##

#merge parcels with use descriptions to get use codes
parcel_data = parcel_data.merge(land_use_lookup, left_on='LUC_Assign_M', right_on='USE_CODE', how='left').groupby(by='LOC_ID').agg('first').reset_index()

#merge real estate lookup codes based on land use codes
parcel_data = parcel_data.merge(real_estate_lookup, left_on='USE_CODE', right_on='use_code_3dg', how='left').groupby(by='LOC_ID').agg('first').reset_index()

#get real estate typology descriptions based on real estate codes
parcel_data = parcel_data.merge(real_estate_lookup_code, on='real_estate_type', how='left').groupby(by='LOC_ID').agg('first').reset_index()

## HEAT FIELD ##
#determine highest heat parcels based on 'rnk_heat' value. high heat in this case means being in the
#top 20% of hottest census blocks

mass_mainland_crs = "EPSG:26986"
parcel_data = parcel_data.set_crs(mass_mainland_crs)
structures_data = structures_data.to_crs(mass_mainland_crs)

parcel_data = heat_score(parcel_data, heat_fp)


heat_bnry_rule = [parcel_data['rnk_heat'] >= 0.8, 
                parcel_data['rnk_heat'] < 0.8]

choices = ['in high heat area', 
            'not in high heat area']

#create a new field based on whether or not a parcel is in a high heat area
parcel_data['heat_mmc'] = np.select(heat_bnry_rule, choices, default=np.nan)

#field for whether or not the parcel is publicly owned
parcel_data = public_ownership(parcel_data)



#reproject all to mass mainland
mass_mainland_crs = "EPSG:26986"
parcel_data = parcel_data.set_crs(mass_mainland_crs)
structures_data = structures_data.to_crs(mass_mainland_crs)

#spatially join structures to parcels, group by structure id to eliminate duplicates
parcel_data = parcel_data.drop(['index_right'], axis=1)
muni_structures_join_parcels = structures_data.sjoin(parcel_data, how='left').groupby(by='STRUCT_ID').agg('first').reset_index()


In [110]:
#environmental justice (2020 boundaries, updated in 2023)
datasets_dir = r'K:\DataServices\datasets'
ej_2020_gdb = os.path.join(datasets_dir, 'Environment and Energy\Environmental_Justice\ej2020.gdb')
ej_2020 = gpd.read_file(ej_2020_gdb, layer='EJ_POLY')
ej_field = 'EJ_CRIT_DESC'



In [ ]:
ej_2020.head()

NameError: name 'ej2020' is not defined

In [ ]:
muni_structures_join_ej = 